In [1]:
import json
import prompt_rag
import torch
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../..", ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import covmis, liar2

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType

# model_type = CustomModelType.mixtral_moe_7b_instruct_awq
model_type = CustomModelType.llama_3_70b_instruct_awq
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    gpu_memory_utilization=0.92,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True, 
)

K = 5
sort = False

prior_knowledge_version = "1"
search_engine = "brave"
model_name = 'llama3'
dataset = 'liar2' # liar2 covmis



[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/llama-3-70b-instruct-awq
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/llama-3-70b-instruct-awq",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "bits": 4,
    "group_size": 128,
    "modules_to_not_convert": null,
    "quant_method": "awq",
    "version": "gemm",
    "zero_point": true
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,

WARNING 07-21 19:53:29 config.py:244] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-21 19:53:29 config.py:698] Defaulting to use mp for distributed inference
INFO 07-21 19:53:29 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='/home/css/models/llama-3-70b-instruct-awq', speculative_config=None, tokenizer='/home/css/models/llama-3-70b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), s

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(VllmWorkerProcess pid=40113) INFO 07-21 19:53:31 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 07-21 19:53:31 utils.py:741] Found nccl from library libnccl.so.2
INFO 07-21 19:53:31 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=40113) INFO 07-21 19:53:31 utils.py:741] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=40113) INFO 07-21 19:53:31 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 07-21 19:53:44 model_runner.py:255] Loading model weights took 18.5481 GB
(VllmWorkerProcess pid=40113) INFO 07-21 19:53:47 model_runner.py:255] Loading model weights took 18.5481 GB
INFO 07-21 19:53:50 distributed_gpu_executor.py:56] # GPU blocks: 785, # CPU blocks: 1638
INFO 07-21 19:53:52 model_runner.py:924] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-21 19:53:52 model_runner.py:928] CUDA g

In [2]:
data_type = 'train' # 只有数据集为liar2时才有效

# data_search = data_search[:10] + [data_search[9690]]
prompt_rag.update_train_search_llm(
    model_name, get_resp_list, search_engine,
    dataset, prior_knowledge_version,
    data_type=data_type, 
    K=K, sort=sort
)


  0%|          | 0/11017 [00:00<?, ?it/s]

WARNING 07-21 19:56:15 scheduler.py:1112] Sequence group 6 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


  3%|▎         | 322/11017 [26:53<16:10:33,  5.44s/it]

WARNING 07-21 20:22:56 scheduler.py:1112] Sequence group 327 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


  6%|▌         | 632/11017 [52:12<10:15:28,  3.56s/it]

...

 97%|█████████▋| 10693/11017 [13:41:52<15:52,  2.94s/it]

WARNING 07-22 09:37:53 scheduler.py:1112] Sequence group 10698 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2051


 99%|█████████▉| 10949/11017 [14:00:15<04:21,  3.84s/it]

WARNING 07-22 09:56:20 scheduler.py:1112] Sequence group 10954 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2101


100%|██████████| 11017/11017 [14:05:04<00:00,  4.60s/it]
